In [112]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Важная настройка для корректной настройки pipeline!
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder, TargetEncoder
from sklearn.model_selection import GridSearchCV, KFold

# for model learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

#models
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.svm import SVC
from catboost import CatBoostRegressor

# Metrics
from sklearn.metrics import *


# tunning hyperparamters model
import optuna

### Подгружаем датасет ###

In [113]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv('/home/zef/DS_Bootcamp/HousePrices/TempData/train.csv')

#Логарифмируем цену
df['SalePrice'] = df['SalePrice'].map(np.log)

In [114]:
#Подгружена заготовленная таблица параметров
PT = pd.read_csv('/home/zef/DS_Bootcamp/HousePrices/ParamTable.csv',index_col=[0])
PT.head(10)

,NaN_count,diff_vals,data_type,drop,NaN_imp_type,encoder,to_scale
Id,0,1460,int64,True,NaN,NaN,True
YearBuilt,0,112,int64,False,NaN,NaN,True
YearRemodAdd,0,61,int64,False,NaN,NaN,True
KitchenAbvGr,0,4,int64,False,NaN,NaN,True
BedroomAbvGr,0,8,int64,False,NaN,NaN,True
HalfBath,0,3,int64,False,NaN,NaN,True
FullBath,0,4,int64,False,NaN,NaN,True
OverallCond,0,9,int64,False,NaN,NaN,True
BsmtHalfBath,0,3,int64,False,NaN,NaN,True
GrLivArea,0,861,int64,False,NaN,NaN,True


In [115]:
#Сплит
X_train, X_valid, y_train, y_valid = train_test_split(df.drop('SalePrice', axis=1), df['SalePrice'], test_size=0.2, random_state=42)

In [116]:
#Первичный отбор признаков на заполнение: делим все где NaN это отдельный класс и где это действительно пропуск (увы, пока вручную)
Cat_With_NaNs = pd.DataFrame(data={'NaN_count': df.isna().sum(), 'Sum':df.count(), 'data_type':df.dtypes})
Cat_With_NaNs = Cat_With_NaNs[(Cat_With_NaNs['NaN_count'] != 0) & (Cat_With_NaNs['data_type'] == 'object')]
NaN_Is_A_Class = Cat_With_NaNs.index.to_list()
NaN_Is_Abcence = [NaN_Is_A_Class.pop(NaN_Is_A_Class.index('Electrical'))]

#результат
NaN_Is_A_Class, NaN_Is_Abcence

(['Alley',
  'MasVnrType',
  'BsmtQual',
  'BsmtCond',
  'BsmtExposure',
  'BsmtFinType1',
  'BsmtFinType2',
  'FireplaceQu',
  'GarageType',
  'GarageFinish',
  'GarageQual',
  'GarageCond',
  'PoolQC',
  'Fence',
  'MiscFeature'],
 ['Electrical'])

In [117]:
#Подставляем в таблицу
for col in NaN_Is_A_Class:
    PT.loc[col,'NaN_imp_type'] = 'constant'

for col in NaN_Is_Abcence:
    PT.loc[col,'NaN_imp_type'] = 'most_frequent'

PT

,NaN_count,diff_vals,data_type,drop,NaN_imp_type,encoder,to_scale
Id,0,1460,int64,True,NaN,NaN,True
YearBuilt,0,112,int64,False,NaN,NaN,True
YearRemodAdd,0,61,int64,False,NaN,NaN,True
KitchenAbvGr,0,4,int64,False,NaN,NaN,True
BedroomAbvGr,0,8,int64,False,NaN,NaN,True
HalfBath,0,3,int64,False,NaN,NaN,True
FullBath,0,4,int64,False,NaN,NaN,True
OverallCond,0,9,int64,False,NaN,NaN,True
BsmtHalfBath,0,3,int64,False,NaN,NaN,True
GrLivArea,0,861,int64,False,NaN,NaN,True


In [118]:
#Определяем тип энкодера - кого больше 2 - target/catboost/(можно и one hot), кого меньше - label
pp_cat = df.select_dtypes('object').nunique(0,dropna=False).to_frame().rename(columns={0:'Uniqie_amount'})
OrdEncCol = pp_cat[pp_cat['Uniqie_amount'] == 2].index.to_list()
TrgEncCol = pp_cat[pp_cat['Uniqie_amount'] > 2].index.to_list() 
OrdEncCol, TrgEncCol

(['Street', 'Utilities', 'CentralAir'],
 ['MSZoning',
  'Alley',
  'LotShape',
  'LandContour',
  'LotConfig',
  'LandSlope',
  'Neighborhood',
  'Condition1',
  'Condition2',
  'BldgType',
  'HouseStyle',
  'RoofStyle',
  'RoofMatl',
  'Exterior1st',
  'Exterior2nd',
  'MasVnrType',
  'ExterQual',
  'ExterCond',
  'Foundation',
  'BsmtQual',
  'BsmtCond',
  'BsmtExposure',
  'BsmtFinType1',
  'BsmtFinType2',
  'Heating',
  'HeatingQC',
  'Electrical',
  'KitchenQual',
  'Functional',
  'FireplaceQu',
  'GarageType',
  'GarageFinish',
  'GarageQual',
  'GarageCond',
  'PavedDrive',
  'PoolQC',
  'Fence',
  'MiscFeature',
  'SaleType',
  'SaleCondition'])

In [119]:
#Подставляем в таблицу
for col in OrdEncCol:
    PT.loc[col,'encoder'] = 'ordinal'

for col in TrgEncCol:
    PT.loc[col,'encoder'] = 'target'

In [120]:
# PT['to_scale'] = False

# for col in X_train.select_dtypes(exclude='object').columns:
#     if df[col].min() < -1 or df[col].max() > 1:
#         PT.loc[col,'to_scale'] = True

# PT.drop(['SalePrice'])
# PT.to_csv('ParamTable.csv')

In [121]:
#Собственно зачем создавалась таблица
PT = pd.read_csv('ParamTable.csv',index_col=[0])
PT

,NaN_count,diff_vals,data_type,drop,NaN_imp_type,encoder,to_scale
Id,0,1460,int64,True,NaN,NaN,True
YearBuilt,0,112,int64,False,NaN,NaN,True
YearRemodAdd,0,61,int64,False,NaN,NaN,True
KitchenAbvGr,0,4,int64,False,NaN,NaN,True
BedroomAbvGr,0,8,int64,False,NaN,NaN,True
HalfBath,0,3,int64,False,NaN,NaN,True
FullBath,0,4,int64,False,NaN,NaN,True
OverallCond,0,9,int64,False,NaN,NaN,True
BsmtHalfBath,0,3,int64,False,NaN,NaN,True
GrLivArea,0,861,int64,False,NaN,NaN,True


In [122]:
drop_features = PT[PT['drop'] == True].index.to_numpy()
PT = PT[PT['drop'] != True]

NaN_most_freq = PT[PT['NaN_imp_type'] == 'most_frequent'].index.to_numpy()
NaN_is_a_class = PT[PT['NaN_imp_type'] == 'constant'].index.to_numpy()
NaN_nums = PT[PT['NaN_imp_type'] == 'mean'].index.to_numpy()
NaN_nums_med = np.array(['GarageYrBlt'])

OrdEncCol = PT[PT['encoder'] == 'target'].index.to_numpy()
TrgEncCol = PT[PT['encoder'] == 'ordinal'].index.to_numpy()

OneHotCol = np.array([])

To_scale = PT[PT['to_scale'] == True].index.to_numpy()

drop_features, NaN_is_a_class, NaN_most_freq, OrdEncCol, TrgEncCol, NaN_nums, To_scale

(array(['Id'], dtype=object),
 array(['BsmtFinType1', 'GarageQual', 'GarageCond', 'Alley', 'PoolQC',
        'Fence', 'MiscFeature', 'GarageFinish', 'GarageType',
        'FireplaceQu', 'BsmtExposure', 'BsmtCond', 'BsmtQual',
        'MasVnrType', 'BsmtFinType2'], dtype=object),
 array(['Electrical'], dtype=object),
 array(['BsmtFinType1', 'Condition1', 'Neighborhood', 'GarageQual',
        'GarageCond', 'PavedDrive', 'LandSlope', 'LotConfig',
        'LandContour', 'LotShape', 'Alley', 'PoolQC', 'Fence',
        'MiscFeature', 'MSZoning', 'SaleType', 'GarageFinish',
        'Condition2', 'GarageType', 'FireplaceQu', 'BsmtExposure',
        'BsmtCond', 'Heating', 'SaleCondition', 'Electrical', 'BsmtQual',
        'Foundation', 'ExterCond', 'ExterQual', 'MasVnrType',
        'Exterior2nd', 'Exterior1st', 'RoofMatl', 'RoofStyle',
        'KitchenQual', 'HouseStyle', 'Functional', 'BldgType',
        'BsmtFinType2', 'HeatingQC'], dtype=object),
 array(['Utilities', 'Street', 'CentralAir']

In [123]:
full_scale = np.concatenate([To_scale,TrgEncCol,NaN_nums_med])
full_scale = np.unique(full_scale)

In [124]:
num_imputer = ColumnTransformer(
    transformers = [
        ('num_imputer_mean', SimpleImputer(strategy='mean'), NaN_nums),
        ('num_imputer_median', SimpleImputer(strategy='median'), NaN_nums_med)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)    

cat_imputer = ColumnTransformer(
    transformers = [
        ('drop_features', 'drop', drop_features), #дроп id
        ('cat_imputer_mf', SimpleImputer(strategy='most_frequent'), NaN_most_freq),
        ('cat_imputer_no_class', SimpleImputer(strategy='constant',fill_value='No_Class'), NaN_is_a_class)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)    

cat_encoder = ColumnTransformer(
    transformers = [
        ('ordinal_encoding', OrdinalEncoder(dtype='int64'), OrdEncCol),
        ('target_encoding',TargetEncoder(target_type='continuous'),TrgEncCol),    
        #('onehot_encoding',OneHotEncoder(sparse_output=False),OneHotCol),    
        ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)    

cat_scaler = ColumnTransformer(
    [
        ('scaling_num_columns', StandardScaler(), full_scale)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)

In [125]:
ml_pipeline = Pipeline(
    [
        ('cat_imputer', cat_imputer),
        ('num_imputer', num_imputer),
        ('cat_encoder', cat_encoder),
        ('cat_scaler', cat_scaler)
        #('model', vc)
    ]
)

temp = ml_pipeline.fit_transform(X_train,y_train)
temp.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,CentralAir,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MSSubClass,MasVnrArea,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,BsmtFinType1,Condition1,Neighborhood,GarageQual,GarageCond,PavedDrive,LandSlope,LotConfig,LandContour,LotShape,Alley,PoolQC,Fence,MiscFeature,MSZoning,SaleType,GarageFinish,Condition2,GarageType,FireplaceQu,BsmtExposure,BsmtCond,Heating,SaleCondition,Electrical,BsmtQual,Foundation,ExterCond,ExterQual,MasVnrType,Exterior2nd,Exterior1st,RoofMatl,RoofStyle,KitchenQual,HouseStyle,Functional,BldgType,BsmtFinType2,HeatingQC
254,0.374235,-0.801923,-0.121008,0.136218,1.037269,-0.285504,1.105320,-0.24287,-0.400282,0.274955,-0.351921,-0.958592,-1.055566,-0.863837,-1.056544,-0.904403,-0.407093,-0.764098,-0.212757,-0.212896,-0.015314,-0.118999,-0.866764,-0.601531,-0.09274,-0.133417,-0.714352,0.372217,-0.820445,-0.070993,-0.275838,0.130203,-0.964566,0.572612,0.216257,1.188402,-0.455469,-1.346063,1.650065,5,2,12,5,5,2,0,4,3,3,1,3,4,1,3,8,2,2,1,3,3,4,1,4,3,4,1,2,3,2,8,8,1,1,3,2,6,0,6,4
1066,-0.958202,0.955088,-0.121008,0.136218,-0.971996,-0.285504,-0.818694,-0.24287,0.511920,0.247771,-0.351921,0.592150,0.773664,-0.456264,0.295092,0.593542,0.083170,1.236947,-0.212757,-0.265245,-0.505211,-0.118999,0.074110,-0.601531,-0.09274,-0.508010,-0.138015,1.268609,-0.088934,-0.070993,-0.275838,-0.973152,0.270755,-0.596547,-1.328543,-0.740157,0.718609,0.439214,0.893677,6,2,8,5,5,2,0,4,3,0,1,3,4,1,3,8,2,2,1,5,3,4,1,4,3,2,2,4,2,2,13,12,1,1,3,5,6,0,6,2
638,-0.965964,-0.801923,-0.121008,-1.106774,-0.971996,-0.285504,-0.818694,-0.24287,0.505196,0.301174,2.290826,-0.958592,-1.055566,-2.257169,-2.408179,0.052618,-1.395250,-0.764098,-0.212757,-0.177841,-0.148922,-0.118999,-0.631546,-0.601531,-0.09274,-0.508010,-0.714352,1.268609,-0.820445,-0.070993,-0.275838,0.484814,-1.582227,-0.603357,0.736966,1.790113,-1.988293,-1.683818,0.137290,6,1,7,3,3,1,0,4,3,3,1,3,2,1,3,8,1,2,6,3,3,4,1,4,0,1,1,4,3,2,14,8,1,1,3,2,6,0,6,2
799,-0.487321,0.989227,-0.121008,0.136218,0.267995,-0.285504,1.105320,-0.24287,-0.915776,0.334195,3.902257,2.142892,-1.055566,-1.119755,-1.056544,-1.653375,0.458975,1.236947,-0.212757,-0.324474,-0.460675,-0.118999,-0.161109,0.859229,-0.09274,-0.133417,-0.714352,1.268609,-0.820445,-0.070993,-0.275838,0.921174,0.270755,-0.750921,1.324821,-0.740157,-1.107734,-1.683818,-0.619098,0,1,18,5,5,2,0,0,3,3,1,3,2,1,3,8,3,2,5,5,3,4,1,4,3,2,0,4,3,1,14,13,1,1,2,0,6,0,6,0
380,-0.370895,0.711564,-0.121008,0.136218,-0.496920,-0.285504,-0.818694,-0.24287,0.532091,0.274955,3.547742,0.592150,0.773664,-0.797488,-1.056544,-2.277519,0.312087,-0.764098,-0.212757,-0.529035,-0.906036,-0.118999,-0.161109,-0.601531,-0.09274,-0.508010,-0.714352,0.372217,-0.820445,-0.070993,-0.275838,0.130203,-0.346905,-0.081209,0.216257,-0.740157,-1.531707,-1.683818,1.650065,3,2,18,5,5,2,0,4,3,3,2,3,4,1,3,8,3,2,5,2,3,4,1,4,3,4,0,4,3,2,14,3,1,1,2,0,6,0,6,4


In [126]:
temp.shape

(1168, 79)

In [127]:
LG = LinearRegression()
LG.fit(temp, y_train)

print('train accuracy:', accuracy_score(y_train, ml_pipeline.predict(X_train))) # Доля правильных ответов на выборке, которая была использована для обучения
print('valid accuracy:', accuracy_score(y_valid, ml_pipeline.predict(X_valid))) # Доля правильных ответов на выборке, которую обученный алгоритм еще не видел

LinearRegression()